# Part 1: Spend analysis

### 📦 Cell 1 — Imports & prep

In [ ]:
# Silence warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from source.db_connect.bigquery_connector import BigQueryConnector
from source.data_processing.analysis_utils import (
    preprocess_detailed_data,
    compute_abc_tiers,
    compute_kmeans_tiers,
    fetch_purchase_data,
)

# Helper for case-insensitive column resolution
def _resolve_column(df, target_name, candidates, synth_func=None, warn_msg=None):
    lower_map = {c.lower(): c for c in df.columns}
    for cand in candidates:
        key = cand.lower()
        if key in lower_map:
            original = lower_map[key]
            if original != target_name:
                df = df.rename(columns={original: target_name})
            return df, True
    # Synthesize if requested
    if synth_func is not None and target_name not in df.columns:
        df[target_name] = synth_func(df)
        if warn_msg:
            print(warn_msg)
        return df, True
    return df, False

# Load data
bq = BigQueryConnector()
df_raw = fetch_purchase_data(bq)

# Preprocess (normalizes names / values)
df = preprocess_detailed_data(df_raw)

# Spend column resolution
SPEND_COL = "total_spend"
if SPEND_COL not in df.columns:
    spend_candidates = [
        "total_spend","purchase_amount_eur","amount_eur","Amount Eur","total_purchase_amount_eur"
    ]
    candidate = next((c for c in spend_candidates if c in df.columns), None)
    if candidate:
        df = df.copy()
        df[SPEND_COL] = pd.to_numeric(df[candidate], errors="coerce").fillna(0)
    elif {"purchase_amount_eur","purchase_quantity"} <= set(df.columns):
        df = df.copy()
        df[SPEND_COL] = (
            pd.to_numeric(df["purchase_amount_eur"], errors="coerce").fillna(0) *
            pd.to_numeric(df["purchase_quantity"], errors="coerce").fillna(0)
        )
    elif {"Amount Eur","Quantity"} <= set(df.columns):
        df = df.copy()
        df[SPEND_COL] = pd.to_numeric(df["Amount Eur"], errors="coerce").fillna(0)
    else:
        raise ValueError(f"Unable to derive spend column. Columns: {df.columns.tolist()}")

# Resolve / construct Class3 when explicit Class3 missing
class3_candidates = [
    "Class3","class3","class_3","Class3Description","class3description","Class3Number","class3number"
]
class3_col = next((c for c in class3_candidates if c in df.columns), None)
if not class3_col:
    generic_class_cols = [c for c in df.columns if c.lower() == "class"]
    if generic_class_cols:
        if len(generic_class_cols) > 1:
            new_cols = list(df.columns)
            seen = {}
            for i, col in enumerate(new_cols):
                if col.lower() == 'class':
                    seen[col] = seen.get(col, 0) + 1
                    if seen[col] > 1:
                        new_cols[i] = f"Class_dup{seen[col]-1}"  # first stays 'Class'
            df.columns = new_cols
            generic_class_cols = [c for c in df.columns if c.lower() in {"class","class_dup0","class_dup1","class_dup2"}]
        ordered = generic_class_cols
        mapping = {}
        if len(ordered) >= 1:
            mapping[ordered[0]] = "Class2" if len(ordered) > 1 else "Class3"
        if len(ordered) >= 2:
            mapping[ordered[1]] = "Class3"
        if len(ordered) >= 3:
            mapping[ordered[2]] = "Class4"
        df = df.rename(columns=mapping)
        if "Class3" not in df.columns:
            synth_source = next((c for c in ["Class2","Class4"] if c in df.columns), None)
            if synth_source:
                df["Class3"] = df[synth_source]
            else:
                raise KeyError(f"Could not synthesize Class3; columns: {df.columns.tolist()}")
    else:
        raise KeyError(f"Could not locate a Class3-like column. Available columns: {df.columns.tolist()}")
else:
    if class3_col != "Class3":
        df = df.rename(columns={class3_col: "Class3"})

# Resolve GroupVendor (case-insensitive)
if "GroupVendor" not in df.columns:
    vendor_candidates = ["GroupVendor","crm_main_group_vendor","group_vendor","Group Vendor","crm_group_vendor"]
    lower_map = {c.lower(): c for c in df.columns}
    for cand in vendor_candidates:
        if cand.lower() in lower_map:
            original = lower_map[cand.lower()]
            if original != "GroupVendor":
                df = df.rename(columns={original: "GroupVendor"})
            break

# Resolve ProductNumber (case-insensitive + synth)
prod_candidates = [
    "ProductNumber","productnumber","product_number","ProductNo","productno","ProductID","productid","ProductId",
    "MaterialNumber","materialnumber","material_number","Material","ItemNumber","itemnumber","item_number"
]
df, _ = _resolve_column(
    df,
    "ProductNumber",
    prod_candidates,
    synth_func=lambda d: "SYN_" + d.index.astype(str),
    warn_msg="Warning: No product identifier column found (case-insensitive). Synthesized 'ProductNumber'."
)

# Resolve ProductDescription (case-insensitive + fallback to ProductNumber)
desc_candidates = [
    "ProductDescription","productdescription","product_description","Description","Product Desc","ProductDesc",
    "MaterialDescription","materialdescription","MaterialDesc","ItemDescription","itemdescription","ItemDesc"
]
df, found_desc = _resolve_column(
    df,
    "ProductDescription",
    desc_candidates,
    synth_func=lambda d: d["ProductNumber"].astype(str),
    warn_msg="Warning: No product description column found. Using ProductNumber as placeholder ProductDescription."
)

# Resolve PurchaseQuantity (case-insensitive + zero synth)
qty_candidates = [
    "PurchaseQuantity","purchasequantity","purchase_quantity","Quantity","Qty","Purchase Qty","purchase_qty","qty"
]
df, found_qty = _resolve_column(
    df,
    "PurchaseQuantity",
    qty_candidates,
    synth_func=lambda d: 0,
    warn_msg="Warning: No purchase quantity column found. Created placeholder 'PurchaseQuantity'=0."
)

# Add ABC tiers
df = compute_abc_tiers(df, spend_col=SPEND_COL, tier_col="abc_tier")

# Analysis year
if "year" in df.columns:
    YEAR = int(pd.to_numeric(df["year"], errors="coerce").dropna().mode().iloc[0])
else:
    YEAR = datetime.now().year

print(f"Rows: {len(df):,} | Cols: {len(df.columns)} | YEAR={YEAR}")
print("Columns:", df.columns.tolist())
print(f"Sample data:\n{df.head()}")

### 📈 Cell 2 — Scatter: distribution of spend per Class3

In [ ]:
# Per-product spend (ensure numeric)
work = df.copy()
work[SPEND_COL] = pd.to_numeric(work[SPEND_COL], errors="coerce").fillna(0)

# Rank within each Class3 to get a nice spread on X
work["rank_in_class3"] = work.groupby("Class3")[SPEND_COL].rank(method="first", ascending=False)

# Determine legend (Class3) ordering by total spend descending
class3_spend = work.groupby("Class3")[SPEND_COL].sum().sort_values(ascending=False)
class3_order = class3_spend.index.tolist()

plt.figure(figsize=(10,6))
sns.scatterplot(
    data=work,
    x="rank_in_class3",
    y=SPEND_COL,
    hue="Class3",
    hue_order=class3_order,
    alpha=0.6,
    s=25
)
plt.yscale("log")  # spend is typically heavy-tailed
plt.title("Spend distribution per Class3 (log scale)")
plt.xlabel("Rank within Class3 (1 = highest spend)")
plt.ylabel("Spend")
plt.legend(title="Class3 (desc spend)", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.show()


### 🏷️ Cell 3 — Scatter: distribution of spend by Group Vendor

In [ ]:
work = df.copy()
work[SPEND_COL] = pd.to_numeric(work[SPEND_COL], errors="coerce").fillna(0)

# Limit legend noise: focus on top N vendors by total spend
N_VENDORS = 10
top_vendors = (work.groupby("GroupVendor")[SPEND_COL]
               .sum()
               .sort_values(ascending=False)
               .head(N_VENDORS)
               .index)
work["VendorTop"] = np.where(work["GroupVendor"].isin(top_vendors), work["GroupVendor"], "Other")

# Rank globally by spend
work["rank_global"] = work[SPEND_COL].rank(method="first", ascending=False)

# Legend ordering: VendorTop descending by total spend, keeping 'Other' last if present
vendor_spend = work.groupby("VendorTop")[SPEND_COL].sum().sort_values(ascending=False)
vendor_order = [v for v in vendor_spend.index if v != "Other"] + (["Other"] if "Other" in vendor_spend.index else [])

plt.figure(figsize=(10,6))
sns.scatterplot(
    data=work,
    x="rank_global",
    y=SPEND_COL,
    hue="VendorTop",
    hue_order=vendor_order,
    alpha=0.6,
    s=25
)
plt.yscale("log")
plt.title(f"Spend distribution by Group Vendor (Top {N_VENDORS})")
plt.xlabel("Global rank (1 = highest spend)")
plt.ylabel("Spend")
plt.legend(title="GroupVendor (desc spend)", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.show()


### 🧮 Cell 4 — Count of products by spend interval

In [ ]:
work = df.copy()
work[SPEND_COL] = pd.to_numeric(work[SPEND_COL], errors="coerce").fillna(0)

# Use log-spaced bins for heavy-tailed spend
min_pos = max(work[SPEND_COL].replace(0, np.nan).min(), 1e-6)
max_val = max(work[SPEND_COL].max(), 1)
BINS = 15
bins = np.geomspace(min_pos, max_val, BINS)

work["spend_bin"] = pd.cut(work[SPEND_COL].clip(lower=min_pos), bins=bins, include_lowest=True)
counts = work["spend_bin"].value_counts().sort_index()

plt.figure(figsize=(12,5))
counts.plot(kind="bar")
plt.title("Count of products by spend interval (log-spaced bins)")
plt.xlabel("Spend interval (EUR)")
plt.ylabel("Count of products")
plt.tight_layout()
plt.show()

# (Optional) show a small table
display(counts.to_frame("count").head(10))


### 🤖 Cell 5 — Cluster products at Class3 level by purchase amount (EUR)

In [ ]:
# Cluster per Class3 on *purchase amount EUR* at product level
# If your SPEND_COL already equals purchase_amount_eur over the window, we can reuse SPEND_COL.
# Otherwise, aggregate directly from purchase_amount_eur.

if "purchase_amount_eur" in df.columns:
    base = (df.groupby(["Class3", "ProductNumber"], as_index=False)["purchase_amount_eur"]
              .sum()
              .rename(columns={"purchase_amount_eur": "amount_eur"}))
else:
    base = (df.groupby(["Class3", "ProductNumber"], as_index=False)[SPEND_COL]
              .sum()
              .rename(columns={SPEND_COL: "amount_eur"}))

# Fit k-means within each Class3 (use 3 clusters by default)
k = 3
out_frames = []
for c3, g in base.groupby("Class3", as_index=False):
    g = g.copy()
    # Clean numeric column
    g["amount_eur"] = pd.to_numeric(g["amount_eur"], errors="coerce")
    # Drop NaNs / non-positive entries before clustering
    g = g[g["amount_eur"].notna() & (g["amount_eur"] > 0)]
    if g.empty:
        continue  # nothing to cluster
    # If fewer rows than k, assign all to tier A and continue
    if len(g) < k:
        g["kmeans_tier"] = "A"
        out_frames.append(g)
        continue
    # Reuse analysis_utils' kmeans on a single column by mapping to expected signature
    g["total_spend"] = g["amount_eur"]
    try:
        g = compute_kmeans_tiers(g, spend_col="total_spend", k=k, log_transform=True, tier_col="kmeans_tier")
    except Exception:
        # fallback: simple quantile cut if sklearn missing or convergence issue
        q = g["amount_eur"].rank(pct=True)
        g["kmeans_tier"] = np.where(q <= 1/3, "A", np.where(q <= 2/3, "B", "C"))
    out_frames.append(g.drop(columns=["total_spend"], errors="ignore"))

if out_frames:
    df_class3_clusters = pd.concat(out_frames, ignore_index=True)
    print(df_class3_clusters.head())
else:
    print("No clusters produced (all Class3 groups empty after filtering).")


### 📤 Cell 6 — Export per Class3 to .xlsx with normalized columns and a top metadata row for

In [ ]:
import os
import re

REQUIRED_COLS = ["ProductNumber", "ProductDescription", "PurchaseQuantity"]
missing = [c for c in REQUIRED_COLS if c not in df.columns]
if missing:
    if "ProductDescription" in missing:
        desc_candidates = [
            "ProductDescription","productdescription","product_description","Description","Product Desc","ProductDesc",
            "MaterialDescription","materialdescription","MaterialDesc","ItemDescription","itemdescription","ItemDesc"
        ]
        desc_col = next((c for c in desc_candidates if c in df.columns), None)
        if desc_col:
            df = df.rename(columns={desc_col: "ProductDescription"})
        else:
            df["ProductDescription"] = df.get("ProductNumber", "").astype(str)
            print("Fallback: Synthesized ProductDescription from ProductNumber.")
    if "PurchaseQuantity" in missing:
        qty_candidates = [
            "PurchaseQuantity","purchasequantity","purchase_quantity","Quantity","Qty","Purchase Qty","purchase_qty","qty"
        ]
        qty_col = next((c for c in qty_candidates if c in df.columns), None)
        if qty_col:
            df = df.rename(columns={qty_col: "PurchaseQuantity"})
        else:
            df["PurchaseQuantity"] = 0
            print("Fallback: Created placeholder PurchaseQuantity=0.")
    missing = [c for c in REQUIRED_COLS if c not in df.columns]
    if missing:
        raise KeyError(f"Missing required columns for export after fallback attempts: {missing}")

output_dir = os.path.join(os.getcwd(), "exports_per_class3")
os.makedirs(output_dir, exist_ok=True)

invalid_pattern = re.compile(r"[^A-Za-z0-9._-]+")

for c3, g in df.groupby("Class3"):
    sub = g.loc[:, REQUIRED_COLS].copy()
    sub["ProductNumber"] = sub["ProductNumber"].astype(str)
    sub["ProductDescription"] = sub["ProductDescription"].astype(str)
    sub["PurchaseQuantity"] = pd.to_numeric(sub["PurchaseQuantity"], errors="coerce")

    meta = pd.DataFrame([{ "ProductNumber": "", "ProductDescription": "", "PurchaseQuantity": YEAR }])
    export_df = pd.concat([meta, sub], ignore_index=True)

    raw_name = str(c3)
    safe_c3 = invalid_pattern.sub("_", raw_name).strip("._") or "Class3"
    if len(safe_c3) > 120:
        safe_c3 = safe_c3[:120]
    path = os.path.join(output_dir, f"ABC_Segmentation_{safe_c3}.xlsx")
    export_df.to_excel(path, index=False)
    print(f"✅ Exported {path} ({len(export_df)-1} rows + metadata row) - original label '{raw_name}'")

# Part 2: Enrich exports with product details

In [ ]:
import inspect
import importlib

import source.data_processing.product_utils as product_utils
importlib.reload(product_utils)  # ensure latest chunking / schema fixes loaded

EXPORT_DIR = os.path.join(os.getcwd(), "exports_per_class3")
excel_paths = sorted(
    os.path.join(EXPORT_DIR, f)
    for f in os.listdir(EXPORT_DIR)
    if f.lower().endswith(".xlsx")
)

if not excel_paths:
    print("No exported Excel files found in exports_per_class3; nothing to enrich.")
else:
    exports_cache = {}
    product_numbers = set()
    for path in excel_paths:
        export_df = pd.read_excel(path, dtype={"ProductNumber": str})
        exports_cache[path] = export_df
        if "ProductNumber" not in export_df.columns:
            continue
        product_series = export_df["ProductNumber"].astype(str).str.strip()
        product_series = product_series.replace({"nan": "", "None": ""})
        product_numbers.update(product_series[product_series != ""].tolist())

    if not product_numbers:
        print("No product numbers found across exports; enrichment skipped.")
    else:
        # Prefer wide mapping (pivoted) first
        candidate_funcs = [
            "get_product_details_mapping",  # wide form (one row per product)
            "fetch_product_details",        # long form (detail_name/detail_value)
            "load_product_details",
            "get_product_details",
            "retrieve_product_details",
            "enrich_with_product_details",
            "enrich_products",
            "load_product_master",
            "fetch_product_master",
        ]
        detail_func = None
        for name in candidate_funcs:
            func = getattr(product_utils, name, None)
            if callable(func):
                detail_func = func
                break
        if detail_func is None:
            for name in dir(product_utils):
                if "product" in name.lower():
                    func = getattr(product_utils, name)
                    if callable(func):
                        detail_func = func
                        break
        if detail_func is None:
            raise AttributeError("Could not locate a product detail fetcher in product_utils.py.")

        sig = inspect.signature(detail_func)
        numbers_list = sorted(product_numbers)
        connector_aliases = {"connector", "conn", "bq", "bq_connector", "client", "source_connector", "bqclient", "bq_client"}
        product_aliases = {"product_numbers", "products", "product_ids", "ids", "items"}
        args, kwargs = [], {}
        provided_connector = False
        provided_numbers = False
        accepts_kwargs = any(p.kind == inspect.Parameter.VAR_KEYWORD for p in sig.parameters.values())

        def _is_connector_name(name: str) -> bool:
            lname = name.lower()
            return lname in connector_aliases or any(x in lname for x in ["bq", "connector", "client"])

        def _is_products_name(name: str) -> bool:
            lname = name.lower()
            return lname in product_aliases or any(x in lname for x in ["product", "items", "ids"])

        for param in sig.parameters.values():
            if param.kind in (inspect.Parameter.POSITIONAL_ONLY, inspect.Parameter.POSITIONAL_OR_KEYWORD):
                if param.default is inspect._empty:  # required positional
                    if _is_connector_name(param.name) and not provided_connector:
                        args.append(bq)
                        provided_connector = True
                    elif _is_products_name(param.name) and not provided_numbers:
                        args.append(numbers_list)
                        provided_numbers = True
                    else:
                        ann = param.annotation
                        if ann is not inspect._empty:
                            try:
                                from source.db_connect.bigquery_connector import BigQueryConnector as _BQC
                                if ann is _BQC and not provided_connector:
                                    args.append(bq)
                                    provided_connector = True
                                    continue
                            except Exception:
                                pass
                        raise TypeError(f"Unsupported required parameter '{param.name}' in {detail_func.__name__}.")
                else:  # optional keyword
                    if _is_connector_name(param.name) and not provided_connector:
                        kwargs[param.name] = bq
                        provided_connector = True
                    elif _is_products_name(param.name) and not provided_numbers:
                        kwargs[param.name] = numbers_list
                        provided_numbers = True
            elif param.kind == inspect.Parameter.KEYWORD_ONLY and param.default is inspect._empty:
                if _is_connector_name(param.name) and not provided_connector:
                    kwargs[param.name] = bq
                    provided_connector = True
                elif _is_products_name(param.name) and not provided_numbers:
                    kwargs[param.name] = numbers_list
                    provided_numbers = True

        if not provided_numbers:
            if any(_is_products_name(p.name) for p in sig.parameters.values()):
                target = next(p.name for p in sig.parameters.values() if _is_products_name(p.name))
                kwargs[target] = numbers_list
                provided_numbers = True
            elif accepts_kwargs:
                kwargs["product_numbers"] = numbers_list
                provided_numbers = True
        if not provided_numbers:
            raise TypeError(f"{detail_func.__name__} must accept product numbers as an argument.")

        if not provided_connector:
            for p in sig.parameters.values():
                if _is_connector_name(p.name):
                    kwargs[p.name] = bq
                    provided_connector = True
                    break
        if not provided_connector:
            for p in sig.parameters.values():
                ann = p.annotation
                if ann is not inspect._empty:
                    try:
                        from source.db_connect.bigquery_connector import BigQueryConnector as _BQC
                        if ann is _BQC:
                            kwargs[p.name] = bq
                            provided_connector = True
                            break
                    except Exception:
                        pass

        try:
            detail_df = detail_func(*args, **kwargs)
        except Exception as exc:
            raise RuntimeError(f"Failed to fetch product details via {detail_func.__name__}: {exc}") from exc

        if detail_df is None:
            raise ValueError(f"{detail_func.__name__} returned None; cannot enrich exports.")
        if not isinstance(detail_df, pd.DataFrame):
            detail_df = pd.DataFrame(detail_df)
        if detail_df.empty:
            print(f"{detail_func.__name__} returned no product details; enrichment skipped.")
        else:
            details = detail_df.copy()
            if details.columns.duplicated().any():
                details = details.loc[:, ~details.columns.duplicated()]
            renames = {}
            for col in details.columns:
                low = col.lower()
                if low in {"productnumber", "product_number", "product_id", "productid"}:
                    renames[col] = "ProductNumber"
            if renames:
                details = details.rename(columns=renames)
            if "ProductNumber" not in details.columns:
                raise KeyError("Product details result must include a 'ProductNumber' column.")
            details["ProductNumber"] = details["ProductNumber"].astype(str).str.strip()
            details["ProductNumber"] = details["ProductNumber"].replace({"nan": "", "None": ""})
            details = details[details["ProductNumber"] != ""]
            if details.empty:
                print("Product details dataframe contains no usable ProductNumber values; skipping enrichment.")
            else:
                # Wide vs long handling: if detail_name present, pivot; else assume already wide.
                if {"detail_name", "detail_value"} <= set(details.columns):
                    # pivot to wide form for merging new attribute columns
                    details = details.pivot_table(
                        index="ProductNumber",
                        columns="detail_name",
                        values="detail_value",
                        aggfunc="first"
                    ).reset_index()
                    details.columns.name = None
                    details.columns = [str(c) for c in details.columns]

                detail_cols = [c for c in details.columns if c != "ProductNumber"]
                enriched_files = 0

                for path, export_df in exports_cache.items():
                    if "ProductNumber" not in export_df.columns:
                        continue
                    prod_series = export_df["ProductNumber"].astype(str).str.strip()
                    prod_series = prod_series.replace({"nan": "", "None": ""})
                    meta_mask = prod_series == ""
                    meta_rows = export_df.loc[meta_mask].copy()
                    data_rows = export_df.loc[~meta_mask].copy()
                    if data_rows.empty:
                        continue
                    data_rows["ProductNumber"] = prod_series[~meta_mask]

                    merged = data_rows.merge(details, on="ProductNumber", how="left", suffixes=("", "_detail"))
                    for col in detail_cols:
                        detail_col = f"{col}_detail"
                        if col in merged.columns and detail_col in merged.columns:
                            merged[col] = merged[col].where(merged[col].notna(), merged[detail_col])
                            merged.drop(columns=detail_col, inplace=True)
                        elif detail_col in merged.columns:
                            merged.rename(columns={detail_col: col}, inplace=True)

                    original_cols = list(export_df.columns)
                    new_cols = [c for c in detail_cols if c not in original_cols]
                    final_cols = original_cols + new_cols

                    merged = merged.reindex(columns=final_cols)
                    if not meta_rows.empty:
                        meta_rows = meta_rows.reindex(columns=final_cols)
                        updated = pd.concat([meta_rows, merged], ignore_index=True)
                    else:
                        updated = merged

                    updated.to_excel(path, index=False)
                    enriched_files += 1
                    print(f"✅ Enriched {os.path.basename(path)} ({len(new_cols)} new columns).")

                if enriched_files == 0:
                    print("No Excel files were updated; no matching product numbers found.")